In [1]:
WEBCAM = True
VIDEO_NAME = "fingers"

In [2]:
import time
import cv2
import tkinter as tk
from PIL import Image, ImageTk

# Initialize the GUI application
root = tk.Tk()
root.title("Hand Gesture Recognition")

# Create a label to display the video frames
video_label = tk.Label(root)
video_label.pack()

# Create a text box to display the gesture name
gesture_name_var = tk.StringVar()
gesture_name_entry = tk.Entry(root, textvariable=gesture_name_var, font=('Arial', 14), state='readonly')
gesture_name_entry.pack()

if WEBCAM:
    # Access the webcam
    cap = cv2.VideoCapture(0)
else:
    cap = cv2.VideoCapture(f'videos/{VIDEO_NAME}.mp4')

def detect_five_fingers(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (35, 35), 0)
    
    # Thresholding to get the hand segment
    _, thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        # Assume the largest contour is the hand
        hand_contour = max(contours, key=cv2.contourArea)
        
        # Approximate the contour to a polygon
        approx = cv2.approxPolyDP(hand_contour, 0.02*cv2.arcLength(hand_contour, True), True)
        
        # Convex Hull
        hull = cv2.convexHull(hand_contour, returnPoints=False)
        defects = cv2.convexityDefects(hand_contour, hull)
        
        # Finger counting
        if defects is not None:
            fingers = 0
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(hand_contour[s][0])
                end = tuple(hand_contour[e][0])
                far = tuple(hand_contour[f][0])
                
                # Apply angle rule to count fingers
                a = ((end[0] - start[0])**2 + (end[1] - start[1])**2)**0.5
                b = ((far[0] - start[0])**2 + (far[1] - start[1])**2)**0.5
                c = ((end[0] - far[0])**2 + (end[1] - far[1])**2)**0.5
                angle = (a**2 + b**2 - c**2) / (2*a*b)
                
                # If angle between fingers is less than 90 degrees, it's considered as a finger
                if angle <= 0.5:
                    fingers += 1
            
            # if fingers == 4:  # We count the valleys (4) for 5 fingers
            #     return "Five fingers"
            return f"{fingers + 1} fingers."
            
    return "No contours of hand"

def show_frames():
    # Capture the current frame
    ret, frame = cap.read()
      
    # Detect the number of fingers
    gesture = detect_five_fingers(frame)
    
    # Update the gesture name variable
    gesture_name_var.set(gesture)
    
    # Convert the image from BGR to RGB
    cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)
    
    # Repeat after an interval to capture continuously
    video_label.after(20, show_frames)


def parse_videos():
    # Capture the current frame

    ret, frame = cap.read()

    while ret:
        # run with the functions we tested before, and write the process frame into the video file
        # image_process = preprocess_image(image)
        # image_detect = draw_detected_cat(image_process)
        # vidwrite.write(image_detect) # write frame into video

        # Detect the number of fingers
        gesture = detect_five_fingers(frame)
    
        # Update the gesture name variable
        gesture_name_var.set(gesture)
    
        # Convert the image from BGR to RGB
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
        video_label.imgtk = imgtk
        video_label.configure(image=imgtk)
    
        # Repeat after an interval to capture continuously
        time.sleep(0.05)
        root.update()
        
        ret, frame = cap.read() # read frame from video
    
# Start the GUI
if WEBCAM:
    show_frames()
else:
    parse_videos()
root.mainloop()

# Release the webcam when the GUI is closed
cap.release()


In [2]:
import cv2
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np

# Initialize the GUI application
root = tk.Tk()
root.title("Hand Gesture Recognition")

# Create a label to display the video frames
video_label = tk.Label(root)
video_label.pack()

# Create a text box to display the gesture name
gesture_name_var = tk.StringVar()
gesture_name_entry = tk.Entry(root, textvariable=gesture_name_var, font=('Arial', 14), state='readonly')
gesture_name_entry.pack()

if WEBCAM:
    # Access the webcam
    cap = cv2.VideoCapture(0)
else:
    cap = cv2.VideoCapture(f'videos/{VIDEO_NAME}.mp4')


def filter_skin(image):
    # Convert to YCrCb color space, which is better for skin color detection
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    # Define lower and upper bounds for skin color
    lower_bound = np.array([0, 133, 77], dtype="uint8")
    upper_bound = np.array([255, 173, 127], dtype="uint8")
    
    # Create a mask for skin color
    mask = cv2.inRange(ycrcb, lower_bound, upper_bound)
    
    # Apply the mask to get the skin region
    skin_region = cv2.bitwise_and(image, image, mask=mask)
    
    return skin_region


def preprocess_for_contours(image):
    # Filter skin color first
    skin_region = filter_skin(image)
    
    # Convert to grayscale
    gray = cv2.cvtColor(skin_region, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (35, 35), 0)
    
    # Thresholding to get the hand segment
    _, thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    return thresh



# Function to detect fist
def detect_fist(image):
    skin_region = filter_skin(image)
    # Convert to grayscale

    gray = cv2.cvtColor(skin_region, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (35, 35), 0)
    
    # Thresholding to get the hand segment
    _, thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        # Assume the largest contour is the hand
        hand_contour = max(contours, key=cv2.contourArea)

        x, y, w, h = cv2.boundingRect(hand_contour)
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Approximate the contour to a polygon
        approx = cv2.approxPolyDP(hand_contour, 0.02*cv2.arcLength(hand_contour, True), True)
        
        # If the number of vertices is small, we might have a fist
        if len(approx) < 5:
            return "Fist",image
    return "Not Fist", image


def detect_five_fingers(image):

    skin_region = filter_skin(image)
    # Convert to grayscale
    gray = cv2.cvtColor(skin_region, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (35, 35), 0)
    
    # Thresholding to get the hand segment
    _, thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        # Assume the largest contour is the hand
        hand_contour = max(contours, key=cv2.contourArea)

        x, y, w, h = cv2.boundingRect(hand_contour)
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
        
        # Approximate the contour to a polygon
        approx = cv2.approxPolyDP(hand_contour, 0.02*cv2.arcLength(hand_contour, True), True)
        
        # Convex Hull
        hull = cv2.convexHull(hand_contour, returnPoints=False)
        defects = cv2.convexityDefects(hand_contour, hull)
        
        # Finger counting
        if defects is not None:
            fingers = 0
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(hand_contour[s][0])
                end = tuple(hand_contour[e][0])
                far = tuple(hand_contour[f][0])
                
                # Apply angle rule to count fingers
                a = ((end[0] - start[0])**2 + (end[1] - start[1])**2)**0.5
                b = ((far[0] - start[0])**2 + (far[1] - start[1])**2)**0.5
                c = ((end[0] - far[0])**2 + (end[1] - far[1])**2)**0.5
                angle = (a**2 + b**2 - c**2) / (2*a*b)
                
                # If angle between fingers is less than 90 degrees, it's considered as a finger
                if angle <= 0.5:
                    fingers += 1
            
            if fingers == 4:  # We count the valleys (4) for 5 fingers
                return "Five fingers", image
    return "Not five fingers", image
# Modify the existing function to handle both fist and five fingers
def detect_gestures(image):
    fist, image_with_box = detect_fist(image)
    if fist == "Fist":
        return "Fist", image_with_box
    
    five_fingers, image_with_box = detect_five_fingers(image)
    if five_fingers == "Five fingers":
        return "Five fingers", image_with_box
    
    return "Unknown gesture", image

# Update the show_frames function to use the new detect_gestures function
def show_frames():
    # Capture the current frame
    ret, frame = cap.read()
    
    # Detect the gesture
    gesture, frame_with_box = detect_gestures(frame)
    
    # Update the gesture name variable
    gesture_name_var.set(gesture)
    
    # Convert the image from BGR to RGB
    cv2image = cv2.cvtColor(frame_with_box, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)
    
    # Repeat after an interval to capture continuously
    video_label.after(20, show_frames)

# Start the GUI
if WEBCAM:
    show_frames()
else:
    parse_videos()
root.mainloop()

# Release the webcam when the GUI is closed
cap.release()


In [25]:
#best one yet at detecting the hand
import cv2
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np

# Initialize the GUI application
root = tk.Tk()
root.title("Fist Detection")

# Create a label to display the video frames
video_label = tk.Label(root)
video_label.pack()

# Create a text box to display the gesture name
gesture_name_var = tk.StringVar()
gesture_name_entry = tk.Entry(root, textvariable=gesture_name_var, font=('Arial', 14), state='readonly')
gesture_name_entry.pack()

# Access the webcam
cap = cv2.VideoCapture(0)

def filter_skin(image):
    # Convert to YCrCb color space
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    # Define lower and upper bounds for skin color
    lower_bound = np.array([0, 133, 77], dtype="uint8")
    upper_bound = np.array([255, 173, 127], dtype="uint8")
    
    # Create a mask for skin color
    mask = cv2.inRange(ycrcb, lower_bound, upper_bound)
    
    # Apply the mask to get the skin region
    skin_region = cv2.bitwise_and(image, image, mask=mask)
    
    return skin_region

# Function to detect fist
def detect_fist(image):
    skin_region = filter_skin(image)
    gray = cv2.cvtColor(skin_region, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (35, 35), 0)
    _, thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on size
    frame_area = image.shape[0] * image.shape[1]
    contours = [cnt for cnt in contours if 0.01 < cv2.contourArea(cnt) / frame_area < 0.1]

    for contour in contours:
        hull = cv2.convexHull(contour)
        if cv2.isContourConvex(hull):
            x, y, w, h = cv2.boundingRect(hull)
            aspect_ratio = w / float(h)
            if 0.75 < aspect_ratio < 1.25:
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                return "Fist Detected", image

    return "No Fist Detected", image



# Function to display frames in the GUI
def show_frames():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        return
    
    # Detect the gesture
    gesture, frame_with_box = detect_fist(frame)
    
    # Update the gesture name variable
    gesture_name_var.set(gesture)
    
    # Convert the image from BGR to RGB
    cv2image = cv2.cvtColor(frame_with_box, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)
    
    # Repeat after an interval to capture continuously
    video_label.after(20, show_frames)

# Start the GUI
show_frames()
root.mainloop()

# Release the webcam when the GUI is closed
cap.release()


In [16]:
#works for fist and open hand

import cv2
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np

# Initialize the GUI application
root = tk.Tk()
root.title("Hand Gesture Detection")

# Create a label to display the video frames
video_label = tk.Label(root)
video_label.pack()

# Create a text box to display the gesture name
gesture_name_var = tk.StringVar()
gesture_name_entry = tk.Entry(root, textvariable=gesture_name_var, font=('Arial', 14), state='readonly')
gesture_name_entry.pack()

# Access the webcam
cap = cv2.VideoCapture(0)

def filter_skin(image):
    # Convert to YCrCb color space
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    # Define lower and upper bounds for skin color
    lower_bound = np.array([0, 133, 77], dtype="uint8")
    upper_bound = np.array([255, 173, 127], dtype="uint8")
    
    # Create a mask for skin color
    mask = cv2.inRange(ycrcb, lower_bound, upper_bound)
    
    # Apply the mask to get the skin region
    skin_region = cv2.bitwise_and(image, image, mask=mask)
    
    return skin_region

def detect_hand_gesture(image):
    skin_region = filter_skin(image)
    gray = cv2.cvtColor(skin_region, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (35, 35), 0)
    _, thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on size
    frame_area = image.shape[0] * image.shape[1]
    contours = [cnt for cnt in contours if 0.01 < cv2.contourArea(cnt) / frame_area < 0.1]

    for contour in contours:
        hull = cv2.convexHull(contour, returnPoints=False)
        if len(hull) > 3:
            defects = cv2.convexityDefects(contour, hull)
            if defects is not None:
                cnt_defects = 0
                for i in range(defects.shape[0]):  # Counting the defects
                    s, e, f, d = defects[i][0]
                    if d > 1000:  # Filter based on defect depth
                        cnt_defects += 1
                
                hull_area = cv2.contourArea(cv2.convexHull(contour))
                solidity = cv2.contourArea(contour) / hull_area
                
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = w / float(h)
                
                if cnt_defects <= 2 and 0.75 < aspect_ratio < 1.25 and solidity > 0.75:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    return "Fist Detected", image
                elif cnt_defects > 2 and solidity < 0.75:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
                    return "Five Fingers Detected", image

    return "No Gesture Detected", image

def show_frames():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        return
    
    # Detect the gesture
    gesture, frame_with_box = detect_hand_gesture(frame)
    
    # Update the gesture name variable
    gesture_name_var.set(gesture)
    
    # Convert the image from BGR to RGB
    cv2image = cv2.cvtColor(frame_with_box, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)
    
    video_label.after(20, show_frames)

# Start the GUI
show_frames()
root.mainloop()

# Release the webcam when the GUI is closed
cap.release()


In [17]:
#index finger is L detection, best code so far

import cv2
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np

# Initialize the GUI application
root = tk.Tk()
root.title("Hand Gesture Detection")

# Create a label to display the video frames
video_label = tk.Label(root)
video_label.pack()

# Create a text box to display the gesture name
gesture_name_var = tk.StringVar()
gesture_name_entry = tk.Entry(root, textvariable=gesture_name_var, font=('Arial', 14), state='readonly')
gesture_name_entry.pack()

# Access the webcam
cap = cv2.VideoCapture(0)

def filter_skin(image):
    # Convert to YCrCb color space
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    # Define lower and upper bounds for skin color
    lower_bound = np.array([0, 133, 77], dtype="uint8")
    upper_bound = np.array([255, 173, 127], dtype="uint8")
    
    # Create a mask for skin color
    mask = cv2.inRange(ycrcb, lower_bound, upper_bound)
    
    # Apply the mask to get the skin region
    skin_region = cv2.bitwise_and(image, image, mask=mask)
    
    return skin_region


def detect_hand_gesture(image):
    skin_region = filter_skin(image)
    gray = cv2.cvtColor(skin_region, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (35, 35), 0)
    _, thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on size
    frame_area = image.shape[0] * image.shape[1]
    contours = [cnt for cnt in contours if 0.01 < cv2.contourArea(cnt) / frame_area < 0.1]

    for contour in contours:
        hull = cv2.convexHull(contour, returnPoints=False)
        if len(hull) > 3:
            defects = cv2.convexityDefects(contour, hull)
            if defects is not None:
                cnt_defects = 0
                extended_fingers = 0
                for i in range(defects.shape[0]):  # Counting the defects
                    s, e, f, d = defects[i][0]
                    start = tuple(contour[s][0])
                    end = tuple(contour[e][0])
                    far = tuple(contour[f][0])
                    
                    a = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                    b = np.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                    c = np.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                    angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  # Cosine theorem
                    
                    if np.degrees(angle) < 90:  # Open angle
                        cnt_defects += 1
                        if d > 10000:  # Using defect depth to infer extended finger
                            extended_fingers += 1
                
                hull_area = cv2.contourArea(cv2.convexHull(contour))
                solidity = cv2.contourArea(contour) / hull_area
                
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = w / float(h)
                
                # Assuming "One Index Finger Up" if exactly one extended finger is detected
                if extended_fingers == 1:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 165, 0), 2)  # Orange bounding box
                    return "One Index Finger Up Detected", image
                elif cnt_defects > 2 and solidity < 0.75:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)  # Blue for open hand
                    return "Five Fingers Detected", image
                elif cnt_defects <= 2 and 0.75 < aspect_ratio < 1.25 and solidity > 0.75:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)  # Green for fist
                    return "Fist Detected", image

    return "No Gesture Detected", image



      
def show_frames():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        return
    
    # Detect the gesture
    gesture, frame_with_box = detect_hand_gesture(frame)
    
    # Update the gesture name variable
    gesture_name_var.set(gesture)
    
    # Convert the image from BGR to RGB
    cv2image = cv2.cvtColor(frame_with_box, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)
    
    video_label.after(20, show_frames)

# Start the GUI
show_frames()
root.mainloop()

# Release the webcam when the GUI is closed
cap.release()


In [20]:
#index finger is L detection, This kind of works, but 5 fingers vs index finger and thumb can both be L

import cv2
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np

# Initialize the GUI application
root = tk.Tk()
root.title("Hand Gesture Detection")

# Create a label to display the video frames
video_label = tk.Label(root)
video_label.pack()

# Create a text box to display the gesture name
gesture_name_var = tk.StringVar()
gesture_name_entry = tk.Entry(root, textvariable=gesture_name_var, font=('Arial', 14), state='readonly')
gesture_name_entry.pack()

# Access the webcam
cap = cv2.VideoCapture(0)

def filter_skin(image):
    # Convert to YCrCb color space
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    # Define lower and upper bounds for skin color
    lower_bound = np.array([0, 133, 77], dtype="uint8")
    upper_bound = np.array([255, 173, 127], dtype="uint8")
    
    # Create a mask for skin color
    mask = cv2.inRange(ycrcb, lower_bound, upper_bound)
    
    # Apply the mask to get the skin region
    skin_region = cv2.bitwise_and(image, image, mask=mask)
    
    return skin_region


def detect_hand_gesture(image):
    skin_region = filter_skin(image)
    gray = cv2.cvtColor(skin_region, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (35, 35), 0)
    _, thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on size
    frame_area = image.shape[0] * image.shape[1]
    contours = [cnt for cnt in contours if 0.01 < cv2.contourArea(cnt) / frame_area < 0.1]

    for contour in contours:
        hull = cv2.convexHull(contour, returnPoints=False)
        if len(hull) > 3:
            defects = cv2.convexityDefects(contour, hull)
            if defects is not None:
                l_shape_detected = False  # Initialize L shape detection flag
                for i in range(defects.shape[0]):
                    s, e, f, d = defects[i][0]
                    start = tuple(contour[s][0])
                    end = tuple(contour[e][0])
                    far = tuple(contour[f][0])
                    
                    a = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                    b = np.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                    c = np.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                    angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  # Cosine theorem
                    
                    if np.degrees(angle) > 80 and np.degrees(angle) < 100 and d > 10000:  # Approximate angle for L shape
                        l_shape_detected = True
                        break  # Exit the loop if L shape is detected

                if l_shape_detected:
                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 165, 0), 2)  # Orange bounding box for L shape
                    return "L Shape Detected", image

                cnt_defects = 0
                for i in range(defects.shape[0]):  # Counting the defects
                    s, e, f, d = defects[i][0]
                    start = tuple(contour[s][0])
                    end = tuple(contour[e][0])
                    far = tuple(contour[f][0])
                    
                    a = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                    b = np.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                    c = np.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                    angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  # Cosine theorem
                    
                    if angle <= np.radians(90):  # Open angle
                        cnt_defects += 1
                
                hull_area = cv2.contourArea(cv2.convexHull(contour))
                solidity = cv2.contourArea(contour) / hull_area
                
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = w / float(h)
                
                if cnt_defects > 2 and solidity < 0.75:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)  # Blue for open hand
                    return "Five Fingers Detected", image
                elif cnt_defects <= 2 and 0.75 < aspect_ratio < 1.25 and solidity > 0.75:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)  # Green for fist
                    return "Fist Detected", image

    return "No Gesture Detected", image



      
def show_frames():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        return
    
    # Detect the gesture
    gesture, frame_with_box = detect_hand_gesture(frame)
    
    # Update the gesture name variable
    gesture_name_var.set(gesture)
    
    # Convert the image from BGR to RGB
    cv2image = cv2.cvtColor(frame_with_box, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)
    
    video_label.after(20, show_frames)

# Start the GUI
show_frames()
root.mainloop()

# Release the webcam when the GUI is closed
cap.release()


In [52]:
#best code so far, works for L, open hand and fist
import cv2
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np

# Initialize the GUI application
root = tk.Tk()
root.title("Hand Gesture Detection")

# Create a label to display the video frames
video_label = tk.Label(root)
video_label.pack()

# Create a text box to display the gesture name
gesture_name_var = tk.StringVar()
gesture_name_entry = tk.Entry(root, textvariable=gesture_name_var, font=('Arial', 14), state='readonly')
gesture_name_entry.pack()

# Access the webcam
cap = cv2.VideoCapture(0)

def filter_skin(image):
    # Convert to YCrCb color space
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    # Define lower and upper bounds for skin color
    lower_bound = np.array([0, 133, 77], dtype="uint8")
    upper_bound = np.array([255, 173, 127], dtype="uint8")
    
    # Create a mask for skin color
    mask = cv2.inRange(ycrcb, lower_bound, upper_bound)
    
    # Apply the mask to get the skin region
    skin_region = cv2.bitwise_and(image, image, mask=mask)
    
    return skin_region


           
def detect_hand_gesture(image):
    skin_region = filter_skin(image)
    gray = cv2.cvtColor(skin_region, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (35, 35), 0)
    _, thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on size
    frame_area = image.shape[0] * image.shape[1]
    contours = [cnt for cnt in contours if 0.01 < cv2.contourArea(cnt) / frame_area < 0.1]

    for contour in contours:
        hull = cv2.convexHull(contour, returnPoints=False)
        if len(hull) > 3:
            defects = cv2.convexityDefects(contour, hull)
            if defects is not None:
                cnt_defects = 0
                for i in range(defects.shape[0]):  # Count all defects for overall hand posture
                    s, e, f, d = defects[i][0]
                    start = tuple(contour[s][0])
                    end = tuple(contour[e][0])
                    far = tuple(contour[f][0])
                    
                    a = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                    b = np.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                    c = np.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                    angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  # Cosine theorem
                    
                    if angle <= np.radians(90):  # Open angle
                        cnt_defects += 1
                
                # L shape detection logic
                l_shape_defects_count = 0
                for i in range(defects.shape[0]):
                    s, e, f, d = defects[i][0]
                    start = tuple(contour[s][0])
                    end = tuple(contour[e][0])
                    far = tuple(contour[f][0])
                    
                    a = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                    b = np.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                    c = np.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                    angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))
                    
                    # Angle criteria for L shape
                    if np.degrees(angle) >= 80 and np.degrees(angle) <= 100:
                        l_shape_defects_count += 1
                
                hull_area = cv2.contourArea(cv2.convexHull(contour))
                solidity = cv2.contourArea(contour) / hull_area
                
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = w / float(h)
                
                # Adjusted criteria for L shape detection
                if l_shape_defects_count == 1 and cnt_defects <= 1:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 165, 0), 2)  # Orange bounding box for L shape
                    return "L Shape Detected", image
                elif cnt_defects > 3 and solidity < 0.75:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)  # Blue for open hand
                    return "Five Fingers Detected", image
                elif cnt_defects <= 2 and 0.75 < aspect_ratio < 1.25 and solidity > 0.75:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)  # Green for fist
                    return "Fist Detected", image

    return "No Gesture Detected", image

      
def show_frames():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        return
    
    # Detect the gesture
    gesture, frame_with_box = detect_hand_gesture(frame)
    
    # Update the gesture name variable
    gesture_name_var.set(gesture)
    
    # Convert the image from BGR to RGB
    cv2image = cv2.cvtColor(frame_with_box, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)
    
    video_label.after(20, show_frames)

# Start the GUI
show_frames()
root.mainloop()

# Release the webcam when the GUI is closed
cap.release()


In [53]:
#best code so far, works for L, peace sign, open hand and fist
import cv2
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np

# Initialize the GUI application
root = tk.Tk()
root.title("Hand Gesture Detection")

# Create a label to display the video frames
video_label = tk.Label(root)
video_label.pack()

# Create a text box to display the gesture name
gesture_name_var = tk.StringVar()
gesture_name_entry = tk.Entry(root, textvariable=gesture_name_var, font=('Arial', 14), state='readonly')
gesture_name_entry.pack()

# Access the webcam
cap = cv2.VideoCapture(0)

def filter_skin(image):
    # Convert to YCrCb color space
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    # Define lower and upper bounds for skin color
    lower_bound = np.array([0, 133, 77], dtype="uint8")
    upper_bound = np.array([255, 173, 127], dtype="uint8")
    
    # Create a mask for skin color
    mask = cv2.inRange(ycrcb, lower_bound, upper_bound)
    
    # Apply the mask to get the skin region
    skin_region = cv2.bitwise_and(image, image, mask=mask)
    
    return skin_region


           
def detect_hand_gesture(image):
    skin_region = filter_skin(image)
    gray = cv2.cvtColor(skin_region, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (35, 35), 0)
    _, thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on size
    frame_area = image.shape[0] * image.shape[1]
    contours = [cnt for cnt in contours if 0.01 < cv2.contourArea(cnt) / frame_area < 0.1]

    for contour in contours:
        hull = cv2.convexHull(contour, returnPoints=False)
        if len(hull) > 3:
            defects = cv2.convexityDefects(contour, hull)
            if defects is not None:
                cnt_defects = 0
                for i in range(defects.shape[0]):  # Count all defects for overall hand posture
                    s, e, f, d = defects[i][0]
                    start = tuple(contour[s][0])
                    end = tuple(contour[e][0])
                    far = tuple(contour[f][0])
                    
                    a = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                    b = np.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                    c = np.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                    angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  # Cosine theorem
                    
                    if angle <= np.radians(90):  # Open angle
                        cnt_defects += 1
                
                # L shape detection logic
                l_shape_defects_count = 0
                peace_sign = 0
                for i in range(defects.shape[0]):
                    s, e, f, d = defects[i][0]
                    start = tuple(contour[s][0])
                    end = tuple(contour[e][0])
                    far = tuple(contour[f][0])
                    
                    a = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                    b = np.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                    c = np.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                    angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))
                    
                    # Angle criteria for L shape
                    if np.degrees(angle) >= 80 and np.degrees(angle) <= 100:
                        l_shape_defects_count += 1
                    elif np.degrees(angle) <= 55:
                        peace_sign+=1
                
                hull_area = cv2.contourArea(cv2.convexHull(contour))
                solidity = cv2.contourArea(contour) / hull_area
                
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = w / float(h)
                
                # Adjusted criteria for L shape detection
                if l_shape_defects_count == 1 and cnt_defects <= 1:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)  # Red bounding box for L shape
                    return "L Shape Detected", image
                elif peace_sign == 1 and cnt_defects <=1:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 165, 0), 2)  # Cyan bounding box for L shape
                    return "Peace sign detected", image
                elif cnt_defects > 3 and solidity < 0.75:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)  # Blue for open hand
                    return "Five Fingers Detected", image
                elif cnt_defects <= 2 and 0.75 < aspect_ratio < 1.25 and solidity > 0.75:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)  # Green for fist
                    return "Fist Detected", image

    return "No Gesture Detected", image

      
def show_frames():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        return
    
    # Detect the gesture
    gesture, frame_with_box = detect_hand_gesture(frame)
    
    # Update the gesture name variable
    gesture_name_var.set(gesture)
    
    # Convert the image from BGR to RGB
    cv2image = cv2.cvtColor(frame_with_box, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)
    
    video_label.after(20, show_frames)

# Start the GUI
show_frames()
root.mainloop()

# Release the webcam when the GUI is closed
cap.release()
